# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [1]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [2]:
#!pip install -U wikipedia
#!pip install langchain_experimental

In [3]:
#from langchain.agents.agent_toolkits import create_python_agent
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(temperature=0, model=llm_model)
load_tools()

In [14]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [6]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [7]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: We need to calculate 25% of 300, which means we need to multiply 300 by 0.25.

Action:
```
{
  "action": "Calculator",
  "action_input": "300*0.25"
}
```


Observation: Answer: 75.0
Thought:We have the answer to the question.

Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

## Wikipedia example

In [8]:
question = "I am working on black hole, but i don't have enough time to compleate this task. Write a blog about it."
result = agent(question) 



> Entering new AgentExecutor chain...
Question: How can I quickly generate a blog post about black holes?
Thought: I need to gather information about black holes and organize it into a coherent blog post.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Black hole"
}
```

Observation: Page: Black hole
Summary: A black hole is a region of spacetime where gravity is so strong that nothing, including light and other electromagnetic waves, has enough energy to escape it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. The boundary of no escape is called the event horizon. Although it has a great effect on the fate and circumstances of an object crossing it, it has no locally detectable features according to general relativity. In many ways, a black hole acts like an ideal black body, as it reflects no light. Moreover, quantum field theory in curved spacetime predicts that event horizons emit Hawking radiation,

## Python Agent

In [15]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [16]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [18]:
agent.run(f"""reome a customer named "Jen","Ayai" from the list: {customer_list}""") 



> Entering new AgentExecutor chain...
I can use the remove() method to remove the specified sublist from the list.
Action: Python_REPL
Action Input: lst = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
lst.remove(['Jen', 'Ayai'])
print(lst)
Observation: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former']]

Thought:The sublist containing "Jen" and "Ayai" has been successfully removed from the list.
Final Answer: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former']]

> Finished chain.


"[['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former']]"

In [24]:
agent.run("explain about python lists by using Cricket")



> Entering new AgentExecutor chain...
I'm not sure what the connection between Python lists and cricket is, but I can explain Python lists in general.
Action: Python_REPL
Action Input: N/A
Observation: NameError("name 'N' is not defined")
Thought:Oops, I accidentally pressed enter without typing anything. Let me try again.
Action: Python_REPL
Action Input: print("Python lists are a type of data structure in Python that can hold multiple values, including integers, strings, and even other lists.")
Observation: Python lists are a type of data structure in Python that can hold multiple values, including integers, strings, and even other lists.

Thought:I still don't see the connection to cricket, but I can give more information about Python lists.
Action: Python_REPL
Action Input: print("Lists in Python are ordered, meaning that the items in the list have a specific order. They are also mutable, which means that you can change the items in the list after it has been created.")
Observati

'Python lists are a versatile data structure in Python that can be used to store and manipulate various types of data, including in a cricket context.'

#### View detailed outputs of the chains

In [25]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

## Define your own tool

In [ ]:
#!pip install DateTime

In [26]:
from langchain.agents import tool
from datetime import date

In [27]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [35]:
@tool
def text_editor(text: str) -> str:
    """your a text editor library capable of editing text \
    with different font types and size. your capable of \
    create a whole blog post without any images."""
    return str(texteditor.EDITOR())

In [31]:
agent= initialize_agent(
    tools + [text_editor], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 


If it does, please try running it again.

In [34]:
import texteditor

#agent("write a blog about anything") 

Reminder: Download your notebook to you local computer to save your work.